In [5]:
import pandas as pd
import requests

In [6]:
#크롤링 할 웹 URL
page_no = 1

page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}'

# requests.get() => url source 크롤링해오기
source = requests.get(page_url).text

In [20]:
# source test 출력

source

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 증권</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script src="https

In [9]:
# beautifulsoup4를 불러온다.

import bs4

In [10]:


source = bs4.BeautifulSoup(source)

In [18]:
#prettify => url source  정갈하게 출력하기

print(source.prettify())

<html lang="ko">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   네이버 증권
  </title>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/common.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/layout.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/main.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock2.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/newstock3.css" rel="stylesheet" type="text/css"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20230321170048/css/world.css" rel="stylesheet" type="text/css"/>
 </head

In [27]:
# find_all()는 HTML source에서 조건을 만족하는 모든 tag를 가져오는 함수 
prices = source.find_all('td', class_='number_1') # class가 number_1인 td 태그 가져오기

In [28]:
dates = source.find_all('td',class_=('date')) #class가 date인 td태그 가져오기

In [25]:
prices[0].text

'333.68'

In [56]:
price_list = [] 

for n in prices[::4]:              #
    price_list.append(n.text)

price_list

['335.05', '333.09', '332.09', '330.87', '326.38', '323.36']

In [66]:
# X-Path : /html/body/div/table[2]/tbody/tr/td[12]/a   태그의 인덱스 값으로 URL 찾을 수 있음

#
last_url = source.find_all("td",class_="pgRR")[0].find_all("a")[0]["href"]
last_url

'/sise/sise_index_day.naver?code=KPI200&page=713'

In [69]:
last_page = int(last_url.split('&page=')[-1])
last_page

713

In [100]:
price_list = []
date_list = []
for page_no in range(1,last_page+1):
    
    page_url = f'https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}'

    source = requests.get(page_url).text
    
    source = bs4.BeautifulSoup(source)
    source.prettify()
    prices = source.find_all('td', class_='number_1')
    dates = source.find_all('td',class_=('date'))
    
    for date in dates:
        date_list.append(date.text)
        
    for n in prices[::4]:
        price_list.append(n.text)
        
print("end")
    

end


In [110]:
len(price_list) 
len(date_list)

4278

In [113]:
df = pd.DataFrame({'date' : date_list, 'price' : price_list})
df

,date,price
0,2023.04.14,333.50
1,2023.04.13,333.09
2,2023.04.12,332.09
3,2023.04.11,330.87
4,2023.04.10,326.38
...,...,...
4273,,
4274,,
4275,,
4276,,


In [115]:
df.to_excel("kpi200.xlsx",index=False)